In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [7]:
X = tf.placeholder(tf.float32, [None, 28, 28,1])
Y = tf.placeholder(tf.float32, [None, 10])

In [12]:
#16 First Layer
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
#padding = 'SAME' keeps output size equal to input
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')

In [13]:
print(L1)

Tensor("Conv2D_3:0", shape=(?, 28, 28, 32), dtype=float32)


In [9]:
#19
L1 = tf.nn.relu(L1)
#Pooling layer
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME') 
# 여기서는 padding 의미 없음/ 홀수 인 경우만 의미 있음
print(L1)

Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)


In [10]:
#20 Second Layer
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME') 
print(L2)

Tensor("MaxPool_3:0", shape=(?, 7, 7, 64), dtype=float32)


In [11]:
#21 3rd Layer
W3 = tf.Variable(tf.random_normal([7*7*64,256], stddev=0.01))
L3 = tf.reshape(L2, [-1,7*7*64]) # -1 : batch size( batch를 몰라서(=None개념))
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Relu_4:0", shape=(?, 256), dtype=float32)


In [14]:
#22 4th layer
W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3, W4)
print(model)

Tensor("MatMul_1:0", shape=(?, 10), dtype=float32)


In [21]:
#23
cost2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer1 = tf.train.AdamOptimizer(0.001).minimize(cost1)
optimizer2 = tf.train.AdamOptimizer(0.001).minimize(cost2)
# 최근에는 주로 AdamOptimizer 사용
print(optimizer1)
print(optimizer2)

name: "Adam_5"
op: "NoOp"
input: "^Adam_5/update_Variable_1/ApplyAdam"
input: "^Adam_5/update_Variable_2/ApplyAdam"
input: "^Adam_5/update_Variable_3/ApplyAdam"
input: "^Adam_5/update_Variable_5/ApplyAdam"
input: "^Adam_5/Assign"
input: "^Adam_5/Assign_1"

name: "Adam_6"
op: "NoOp"
input: "^Adam_6/update_Variable_1/ApplyAdam"
input: "^Adam_6/update_Variable_2/ApplyAdam"
input: "^Adam_6/update_Variable_3/ApplyAdam"
input: "^Adam_6/update_Variable_5/ApplyAdam"
input: "^Adam_6/Assign"
input: "^Adam_6/Assign_1"



In [23]:
#24
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [24]:
#25
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [28]:
for epoch in range(10):
    total_cost1 = 0
    total_cost2 = 0
    
    for i in range(total_batch) : 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        
        _1,cost_val1 = sess.run( [optimizer1, cost1],
                              feed_dict={X : batch_xs, Y : batch_ys})
        _2,cost_val2 = sess.run( [optimizer2, cost2],
                              feed_dict={X : batch_xs, Y : batch_ys})
        
        total_cost1 += cost_val1
        total_cost2 += cost_val2
    
    print('Epoch:', '%04d' %(epoch +1),
         'Avg. cost = ','{:,.3f}'.format(total_cost1/total_batch))
    print('Epoch:', '%04d' %(epoch +1),
         'Avg. cost = ','{:,.3f}'.format(total_cost2/total_batch))
    
        
        

Epoch: 0001 Avg. cost =  0.250
Epoch: 0001 Avg. cost =  0.231
Epoch: 0002 Avg. cost =  0.070
Epoch: 0002 Avg. cost =  0.061
Epoch: 0003 Avg. cost =  0.047
Epoch: 0003 Avg. cost =  0.040
Epoch: 0004 Avg. cost =  0.036
Epoch: 0004 Avg. cost =  0.029
Epoch: 0005 Avg. cost =  0.029
Epoch: 0005 Avg. cost =  0.022
Epoch: 0006 Avg. cost =  0.023
Epoch: 0006 Avg. cost =  0.017
Epoch: 0007 Avg. cost =  0.020
Epoch: 0007 Avg. cost =  0.014
Epoch: 0008 Avg. cost =  0.015
Epoch: 0008 Avg. cost =  0.010
Epoch: 0009 Avg. cost =  0.013
Epoch: 0009 Avg. cost =  0.008
Epoch: 0010 Avg. cost =  0.011
Epoch: 0010 Avg. cost =  0.006


In [ ]:
corectness = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(corectness, tf.float32))
print('Accuracy', sess.run(accuracy,
                          feed_dict={
                              X : mnist.test.image.reshape(-1,28,28,1),
                              Y : mnist.test.labels}))

(RawData / image) 28X28X1

(1stLayer)>>>>>> 14X14x32

(2ndLayer)>>>>>> 7X7x64

(3rdLayer)>>>>>> batchX (7x7x64) >>> batchX256 

(4thLayer)>>>>>> 256 X 10 >>> batch *10
